# Capella API: Task and Status Check Example

In [27]:
# Required libraries:
# requests
# json

Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.

{"username": "yourusername","password": "xxxxxxxxx"}

### Import required libraries, build a print utility function, assign API endpoints and load Credentials

In [28]:
import requests
import json
from datetime import datetime, timedelta

# Capella API endpoints
URL = 'https://api.capellaspace.com'
token = '/token'
task = '/task'
collects_list = '/collects/list/'
collects = '/collects/'

#Load username and password
with open('../credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']
username

'david.hemphill@capellaspace.com'

### Get and Print Access Token

In [29]:
#Get the token
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))

accesstoken = r.json()["accessToken"]

# Print the token
print("Access Token: " + accesstoken)

headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}

Access Token: eyJraWQiOiJCeFdcL0tmZ0QzK3pwVlRBSFZ2NlBnNXRtWjVpOXRYTm8zaDkycFwvT0VONXc9IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJhMTkzMWQ5ZS0xNTY0LTQ2ZDUtYjEwZi1hZGE0NDQyYWNlZDQiLCJldmVudF9pZCI6IjIwNzY3OTIxLTNlYjYtNDZkMC05N2MzLTY1ZDVjZjdjNzUyMCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE2NTE2MzExMjYsImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy13ZXN0LTIuYW1hem9uYXdzLmNvbVwvdXMtd2VzdC0yXzJmNFBGWThoOCIsImV4cCI6MTY1MTYzNDcyNSwiaWF0IjoxNjUxNjMxMTI2LCJqdGkiOiJmN2FhZGNhMC01YTFmLTRmNmYtYWNlNi02Mjk1NjNlYTkwNmYiLCJjbGllbnRfaWQiOiIxMGlxZTdiZmp2a2lhMnBzdG9vNzhqaWo5MSIsInVzZXJuYW1lIjoiYzJlYTg4MzEtNmFjYy00ZWM0LWFmZGQtNWUxYjU0N2U5ZWYxIn0.KR9tEvo3MTVF050jeSNnie750QFLrmk89pYvzq2zPWPkNHFM2spuhsSILGctC5QuzICwW1cFatxGvNiul29cNQPmxsHqwziDtAWC25BsJHLHoNO8jY8-yVSbr7_coXJR6HZVjZwH_sI2uUStX3f_NqS1oI4Vz9q0mL1yvE4MCIrbCiLGMepilJxacL1UipYZpSEQah3BFQj8xW86aqwqwoYwSTQtog-H-fFqTd0sys-IOWBl6TZnKop1uXJrJdOVqzLNMFqapY2RJcPN3hPr4pEqXokVjCALHP7Aw8kgCt8odJut_xzIJnutxvsHqsVUBS2

In [30]:
### Is tasked Accepted or Completed (or Rejected/Failed)

In [31]:
# This is to check the tasking status and obtain a collect id.
# If you already have a collect_id, you may proceed to the 
# "Check collect status" section below

#Set a tasking id
taskingrequestId = 'abbddade-888b-4f03-8d55-b08365cac4dd'

In [32]:
# Check the task status - run this as desired
#
# Possible status:
#
# received   : Request has been received by the system for cost estimation and/or mission awareness. 
#              at this stage is has not been submitted for scheduling.
# review     : task is awaiting review before submitting
# submitted  : task is awaiting scheduling (runs every 15 minutes)
# active     : Request is attempting to be scheduled, but we cannot yet guarantee full completion at this time 
#              no opportunity found yet and/or close_window of task is not within the scheduling horizon
# accepted   : task is accepted for collect and committed to collection window
# rejected   : task cannot be collected within the specified window
# completed  : task is completed and collect delivered to catalog
# anomaly    : An anomaly occurred during collect that prevents full completion of the tasking request
# expired    : We have been able to determine that by the end of the request we 
#              will not be able to fully collect AOI.	
# canceled   : Customer canceled the task - will not be collected
# error      : An error occurred during processing and ingestion of the request

call_url = URL + task + "/" + taskingrequestId
print("Calling %s" % call_url)
r = requests.get(call_url, headers=headers)
response = r.json()

#Current status is at the front of the array
statusHistory = response["properties"]["statusHistory"]
current_status = statusHistory[0]
current_status

Calling https://api.capellaspace.com/task/abbddade-888b-4f03-8d55-b08365cac4dd


{'time': '2022-05-04T01:46:07.215Z',
 'code': 'accepted',
 'message': 'Request can be completely satisfied during validity window.'}

### Get the collect id

In [36]:
# After a task is accepted, it is assigned a collect id
# IMPORTANT: 
#    The collect id becomes the primary reference id to find 
#    it in catalog after it is collected - and to place an order for download
# Once the task is "accepted" the collect life cycle takes over until the task is completed or fails due to anomaly

call_url = URL + collects_list + taskingrequestId
print("Calling %s" % call_url)
r = requests.get(call_url, headers=headers)
response = r.json()
collectId = "No collect id available until task is accepted."
if len(response) > 0: 
    collectId = response[0]['collectId'] # only 1 collect expected
collectId

Calling https://api.capellaspace.com/collects/list/abbddade-888b-4f03-8d55-b08365cac4dd


'f67a23a2-fb60-4794-b887-c1139e2b9d34'

### Check collect status

In [37]:
# After a task is accepted, each collect has it's own lifecycle inside of the task request
# It is possible for a collect to be rescheduled inside the tasking window
# Also, for area collects, there are multiple collects within the single tasking request
# Use this call to monitor progress for specific progress. Key status are defined below:
#
# predicted : task is committed for collect within the open/close window with a predicted collect time
# tasked    : task is aboard the spacecraft waiting for collect
# collected : task is collected but still on board the spacecraft waiting for downlink to ground station
# processing: collect is downlinked and being processed
# qa        : collect is in the QA check phase
# delivered : collect is delivered an accessible from catalog
# anomaly   : anomaly occurred, collect not delivered

call_url = URL + collects + collectId
print("Calling %s" % call_url)
r = requests.get(call_url, headers=headers)
response = r.json()
collectStatusHistory = response['collectStatusHistory']
status = collectStatusHistory[0]['code']
if 'predicted' == status:
    details = response['windowOpen']
print("%s: %s" % (status, details))

Calling https://api.capellaspace.com/collects/f67a23a2-fb60-4794-b887-c1139e2b9d34
tasked: 2022-05-04T12:35:11.233Z


In [38]:
response

{'center': [-46.29031399997777, -23.969320343167823, 4.643904152367414],
 'centerEcef': [4029477.890316235, -4215180.877762907, -2575180.695444918],
 'spacecraftId': 3,
 'collectId': 'f67a23a2-fb60-4794-b887-c1139e2b9d34',
 'tileId': 'f05b4e5c-4b42-4f68-8860-a2c3092858b4',
 'tileGroupId': '91d5fd28-f1ba-4fb2-95bc-ba3e9c88d7ec',
 'taskingrequestId': 'abbddade-888b-4f03-8d55-b08365cac4dd',
 'repeatrequestId': None,
 'windowOpen': '2022-05-04T12:35:10.996Z',
 'windowClose': '2022-05-04T12:35:32.139Z',
 'windowDuration': 21.142661,
 'accessProperties': {'ascdsc': 'descending',
  'lookDirection': 'left',
  'localTime': 34193,
  'azimuthOpen': 298.05131973427416,
  'azimuthClose': 263.92075335537294,
  'elevationMin': 61.431043,
  'elevationMax': 62.584245,
  'offNadirMin': 25.216787,
  'offNadirMax': 26.261705},
 'collectProperties': {'collectDuration': 21.142661721302865,
  'imageLength': 5000,
  'imageWidth': 5000,
  'grr': 0.6511493077515484,
  'azr': 0.5,
  'bandwidth': None,
  'nesz': 